# **TASK**
From the following dataset: *https://storage.googleapis.com/dqlab-dataset/retail_raw_test.csv*, determine:
*   Read dataset.
*   The data type is changed to the correct type:
 *   **customer_id** from string to int64
 *   **quantity** from string to int64
 *   **item_price** from string to int64
*   Transform **product_value** so that it has a uniform shape with 'PXXXX' format, then assign it to a new column **product_id**. If there is 'NaN' or 'NULL' value, then replace it with 'unknown'. Also, drop the **product_value** column.
*   Transform **order_date** into value in the 'YYYY-mm-dd' format.
*   Check for missing data from each column and then fill in the *missing values*
 *   In **brand** with 'no_brand'
 *   In **city** and **province** with 'unknown'
*   Create column **city/province** from combined **city** and **province**
*   Create index based on **city_provice**, **order_date**, **customer_id**, **order_id**, **product_id**
*   Create a **total_price** column as a result of multiplying **quantity** by **item_price**
*   Slice data for Jan-2019 only

    




In [28]:
import pandas as pd

## [1] Read Dataset

In [29]:
df = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/retail_raw_test.csv', low_memory=False)
df.head()

,order_id,order_date,customer_id,city,province,brand,quantity,item_price,product_value
0,1730350,"Dec 11, 2019",'13447,Surakarta,Jawa Tengah,BRAND_F,'24,'113000,1374.0
1,1677490,"Jul 31, 2019",'0,NaN,NaN,BRAND_F,'1,'1164000,1370.0
2,1704211,"Oct 18, 2019",'16128,Jakarta Pusat,DKI Jakarta,BRAND_H,'12,'747000,1679.0
3,1679695,"Aug 07, 2019",'16225,Yogyakarta,Yogyakarta,BRAND_H,'6,'590000,1708.0
4,1679080,"Aug 05, 2019",'0,NaN,NaN,BRAND_E,'2,'740000,1201.0


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       5000 non-null   int64  
 1   order_date     5000 non-null   object 
 2   customer_id    5000 non-null   object 
 3   city           3802 non-null   object 
 4   province       3802 non-null   object 
 5   brand          4995 non-null   object 
 6   quantity       5000 non-null   object 
 7   item_price     5000 non-null   object 
 8   product_value  4995 non-null   float64
dtypes: float64(1), int64(1), object(7)
memory usage: 351.7+ KB


## [2] Change Data Type

In [31]:
df["customer_id"] = df["customer_id"].apply(lambda x: x.split("'")[1]).astype("int64")
df["quantity"] = df["quantity"].apply(lambda x: x.split("'")[1]).astype("int64")
df["item_price"] = df["item_price"].apply(lambda x: x.split("'")[1]).astype("int64")
df.dtypes

order_id           int64
order_date        object
customer_id        int64
city              object
province          object
brand             object
quantity           int64
item_price         int64
product_value    float64
dtype: object

## [3] Data Transformation (*product_value*)

In [32]:
import math
def impute_product_value(val):
    if math.isnan(val):
        return 'unknown'
    else:
        return 'P' + '{:0>4}'.format(str(val).split('.')[0])

df["product_id"] = df["product_value"].apply(lambda x: impute_product_value(x))
df.drop(["product_value"], axis=1, inplace=True)
df.head()

,order_id,order_date,customer_id,city,province,brand,quantity,item_price,product_id
0,1730350,"Dec 11, 2019",13447,Surakarta,Jawa Tengah,BRAND_F,24,113000,P1374
1,1677490,"Jul 31, 2019",0,NaN,NaN,BRAND_F,1,1164000,P1370
2,1704211,"Oct 18, 2019",16128,Jakarta Pusat,DKI Jakarta,BRAND_H,12,747000,P1679
3,1679695,"Aug 07, 2019",16225,Yogyakarta,Yogyakarta,BRAND_H,6,590000,P1708
4,1679080,"Aug 05, 2019",0,NaN,NaN,BRAND_E,2,740000,P1201


## [4] Data Transformation (*order_date*)

In [33]:
months_dict = {
   "Jan":"01",
   "Feb":"02",
   "Mar":"03",
   "Apr":"04",
   "May":"05",
   "Jun":"06",
   "Jul":"07",
   "Aug":"08",
   "Sep":"09",
   "Oct":"10",
   "Nov":"11",
   "Dec":"12"
}
df["order_date"] = pd.to_datetime(df["order_date"].apply(lambda x: str(x)[-4:] + "-" + months_dict[str(x)[:3]] + "-" + str(x)[4:7]))
df.dtypes

order_id                int64
order_date     datetime64[ns]
customer_id             int64
city                   object
province               object
brand                  object
quantity                int64
item_price              int64
product_id             object
dtype: object

## [5] Handling Missing Value

In [34]:
df[["city","province"]] = df[["city","province"]].fillna("unknown")
df["brand"] = df["brand"].fillna("no_brand")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     5000 non-null   int64         
 1   order_date   5000 non-null   datetime64[ns]
 2   customer_id  5000 non-null   int64         
 3   city         5000 non-null   object        
 4   province     5000 non-null   object        
 5   brand        5000 non-null   object        
 6   quantity     5000 non-null   int64         
 7   item_price   5000 non-null   int64         
 8   product_id   5000 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 351.7+ KB


## [6] Create New Column (*city/province*)

In [35]:
df["city/province"] = df["city"] + "/" + df["province"]
df.drop(["city","province"], axis=1, inplace=True)
df.head()


,order_id,order_date,customer_id,brand,quantity,item_price,product_id,city/province
0,1730350,2019-12-11,13447,BRAND_F,24,113000,P1374,Surakarta/Jawa Tengah
1,1677490,2019-07-31,0,BRAND_F,1,1164000,P1370,unknown/unknown
2,1704211,2019-10-18,16128,BRAND_H,12,747000,P1679,Jakarta Pusat/DKI Jakarta
3,1679695,2019-08-07,16225,BRAND_H,6,590000,P1708,Yogyakarta/Yogyakarta
4,1679080,2019-08-05,0,BRAND_E,2,740000,P1201,unknown/unknown


## [7] Create Hierarchical Index

In [36]:
df = df.set_index(["city/province","order_date","customer_id","order_id","product_id"])
df = df.sort_index()
df.head(20)

brand  \
city/province          order_date customer_id order_id product_id            
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936       BRAND_K   
                       2019-11-12 12360       1715116  P0758       BRAND_C   
                                                       P3042       BRAND_R   
                       2019-12-09 12374       1729036  P1660       BRAND_G   
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628       BRAND_C   
                       2019-02-07 12578       1623346  P0782       BRAND_C   
                       2019-11-18 12594       1717372  P1311       BRAND_F   
                                                       P3249       BRAND_S   
                       2019-11-24 12584       1720096  P2923       BRAND_R   
                       2019-12-13 12578       1731949  P3338       BRAND_S   
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597       BRAND_G   
                       2019-01-10 17392       1617952  P2137       BRAND_M   
                       2019-01-14 15527       1618828  P3115       BRAND_S   
                       2019-01-29 13253       1620289  P0099       BRAND_A   
                       2019-02-05 15518       1622449  P2426       BRAND_P   
                                  17569       1622482  P2157       BRAND_M   
                       2019-02-13 14572       1624813  P1678       BRAND_H   
                                                       P1794       BRAND_H   
                       2019-02-15 17890       1625137  P3909       BRAND_V   
                       2019-02-19 13158       1625986  P0519       BRAND_B   

                                                                   quantity  \
city/province          order_date customer_id order_id product_id             
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936             24   
                       2019-11-12 12360       1715116  P0758              8   
                                                       P3042             12   
                       2019-12-09 12374       1729036  P1660              4   
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628             12   
                       2019-02-07 12578       1623346  P0782             12   
                       2019-11-18 12594       1717372  P1311              6   
                                                       P3249             12   
                       2019-11-24 12584       1720096  P2923              1   
                       2019-12-13 12578       1731949  P3338             16   
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597              9   
                       2019-01-10 17392       1617952  P2137              2   
                       2019-01-14 15527       1618828  P3115              1   
                       2019-01-29 13253       1620289  P0099             12   
                       2019-02-05 15518       1622449  P2426              3   
                                  17569       1622482  P2157              2   
                       2019-02-13 14572       1624813  P1678              4   
                                                       P1794             12   
                       2019-02-15 17890       1625137  P3909              1   
                       2019-02-19 13158       1625986  P0519              8   

                                                                   item_price  
city/province          order_date customer_id order_id product_id              
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936           450000  
                       2019-11-12 12360       1715116  P0758           695000  
                                                       P3042           310000  
                       2019-12-09 12374       1729036  P1660          2795000  
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628           695000  
                       2019-02-07 12578       1623346

## [8] Create New Column (*total_price*)

In [39]:
df["total_price"] = df["quantity"] * df["item_price"]
df.head()

brand  \
city/province          order_date customer_id order_id product_id            
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936       BRAND_K   
                       2019-11-12 12360       1715116  P0758       BRAND_C   
                                                       P3042       BRAND_R   
                       2019-12-09 12374       1729036  P1660       BRAND_G   
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628       BRAND_C   

                                                                   quantity  \
city/province          order_date customer_id order_id product_id             
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936             24   
                       2019-11-12 12360       1715116  P0758              8   
                                                       P3042             12   
                       2019-12-09 12374       1729036  P1660              4   
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628             12   

                                                                   item_price  \
city/province          order_date customer_id order_id product_id               
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936           450000   
                       2019-11-12 12360       1715116  P0758           695000   
                                                       P3042           310000   
                       2019-12-09 12374       1729036  P1660          2795000   
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628           695000   

                                                                   total_price  
city/province          order_date customer_id order_id product_id               
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936          10800000  
                       2019-11-12 12360       1715116  P0758           5560000  
                                                       P3042           3720000  
                       2019-12-09 12374       1729036  P1660          11180000  
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628           8340000

## [9] Slice Dataset For Only Januari 2019

In [43]:
idx = pd.IndexSlice
df_jan2019 = df.loc[idx[:, "2019-01-01":"2019-01-31"], :]
df_jan2019

brand  \
city/province          order_date customer_id order_id product_id            
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628       BRAND_C   
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597       BRAND_G   
                       2019-01-10 17392       1617952  P2137       BRAND_M   
                       2019-01-14 15527       1618828  P3115       BRAND_S   
                       2019-01-29 13253       1620289  P0099       BRAND_A   
...                                                                    ...   
unknown/unknown        2019-01-30 0           1620766  P3070       BRAND_R   
                                                       P3483       BRAND_S   
                       2019-01-31 0           1621057  P1298       BRAND_F   
                                                       P1773       BRAND_H   
                                                       P2877       BRAND_R   

                                                                   quantity  \
city/province          order_date customer_id order_id product_id             
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628             12   
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597              9   
                       2019-01-10 17392       1617952  P2137              2   
                       2019-01-14 15527       1618828  P3115              1   
                       2019-01-29 13253       1620289  P0099             12   
...                                                                     ...   
unknown/unknown        2019-01-30 0           1620766  P3070              1   
                                                       P3483              3   
                       2019-01-31 0           1621057  P1298              1   
                                                       P1773              5   
                                                       P2877              1   

                                                                   item_price  \
city/province          order_date customer_id order_id product_id               
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628           695000   
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597           520000   
                       2019-01-10 17392       1617952  P2137          1062000   
                       2019-01-14 15527       1618828  P3115          1045000   
                       2019-01-29 13253       1620289  P0099           450000   
...                                                                       ...   
unknown/unknown        2019-01-30 0           1620766  P3070           593000   
                                                       P3483           593000   
                       2019-01-31 0           1621057  P1298           296000   
                                                       P1773           593000   
                                                       P2877          1486000   

                                                                   total_price  
city/province          order_date customer_id order_id product_id               
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628           8340000  
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597           4680000  
                       2019-01-10 17392       1617952  P2137           2124000  
                       2019-01-14 15527       1618828  P3115           1045000  
                       2019-01-29 13253       1620289  P0099           5400000  
...                                                                        ...  
unknown/unknown        2019-01-30 0           1620766  P3070            593000  
                                                       P3483           1779000  
                       2019-01-31 0           1621057  P1298            296000  
                                                       P1773           2965000  
